<a href="https://colab.research.google.com/github/remre/StriveSchool-ai/blob/main/m7NLP/d4/notes/NLPDLAIW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os 
from tensorflow import keras
import zipfile
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer

from datetime import datetime
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
print(tf.__version__)#check version

2.6.0


In [ ]:
os.environ["KAGGLE_CONFIG_DIR"] = ("/content")

In [ ]:
!kaggle datasets download -d 'lakshmi25npathi/imdb-dataset-of-50k-movie-reviews'

 35% 9.00M/25.7M [00:00<00:00, 54.5MB/s]
100% 25.7M/25.7M [00:00<00:00, 85.3MB/s]


In [ ]:
with zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
with open('/content/IMDB Dataset.csv', 'r') as f:
  lines =   f.readlines()
  print(lines)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteM1ZPVT/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteM1ZPVT/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_d, test_d = imdb['train'],  imdb['test']#split the data

In [ ]:
import numpy as np

In [ ]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

start_time = datetime.now()

for s,l in train_d:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  end_time = datetime.now()
  print('Duration: {}'.format(end_time - start_time))
for s,l in test_d:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  

In [ ]:
training_labels = np.array(training_labels)#converting them into np arrays
testing_labels = np.array(testing_labels)

In [ ]:
#some numbers
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok  = '<OOV>'


tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen= max_length, truncating=trunc_type)

testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen= max_length,truncating=trunc_type)



In [ ]:
model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                tf.keras.layers.Flatten(),#could be used also GlobalAvaragePooling1D()
                tf.keras.layers.Dense(6,activation='relu'),
                tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded,
          training_labels,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels))

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.4917 - accuracy: 0.7438 - val_loss: 0.3455 - val_accuracy: 0.8486
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2350 - accuracy: 0.9112 - val_loss: 0.3737 - val_accuracy: 0.8371
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0877 - accuracy: 0.9777 - val_loss: 0.4597 - val_accuracy: 0.8231
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0207 - accuracy: 0.9978 - val_loss: 0.5383 - val_accuracy: 0.8229
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0046 - accuracy: 0.9998 - val_loss: 0.6040 - val_accuracy: 0.8230
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.6357 - val_accuracy: 0.8281
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 8.0388e-04 - accuracy: 1.0000 - val_loss: 0.6815 - val_accuracy: 0.8269
Ep

In [ ]:
e = model.layers[0]

weights = e.get_weights()[0]

print(weights.shape)

(10000, 16)


In [ ]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

In [ ]:
def decode_reviews(text):
  return '  '.join([reverse_word_index.get(i , '?') for i in text])
print(decode_reviews(padded[0]))
print(training_sentences[0])

?  ?  b  this  was  an  absolutely  terrible  movie  don't  be  <OOV>  in  by  christopher  walken  or  michael  <OOV>  both  are  great  actors  but  this  must  simply  be  their  worst  role  in  history  even  their  great  acting  could  not  redeem  this  movie's  ridiculous  storyline  this  movie  is  an  early  nineties  us  propaganda  piece  the  most  pathetic  scenes  were  those  when  the  <OOV>  rebels  were  making  their  cases  for  <OOV>  maria  <OOV>  <OOV>  appeared  phony  and  her  pseudo  love  affair  with  walken  was  nothing  but  a  pathetic  emotional  plug  in  a  movie  that  was  devoid  of  any  real  meaning  i  am  disappointed  that  there  are  movies  like  this  ruining  <OOV>  like  christopher  <OOV>  good  name  i  could  barely  sit  through  it
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acti

In [ ]:
import io


out_v = io.open('vects.tsv','w', encoding= 'utf-8')
out_m  = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + '\n') #
  out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')#value of each item


out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vects.tsv')
  files.download("meta.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
out_v

<_io.TextIOWrapper name='vects.tsv' mode='w' encoding='utf-8'>

In [ ]:
type(training_labels)

numpy.ndarray

In [ ]:
training_labels

array([0, 0, 0, ..., 0, 0, 1])